1. Primero colocar los barcos con las adyacencias
2. Luego es un sistema de turnos: Se van cambiando, jugador/maquina, cuando le toca a la maquina dispara automaticamente (con randoms), cuando le toca al jugador se dispara con inputs, donde quieres tirar? 0,0, en el tal
3. Y gestion de si has acertado en un barco repites turno tanto tu como la maquina
4. Llevar recuento de los barcos que quedan, cuando tu le das a un barco y lo tocas, hay que marcar como que esa coordenada ya esta tocada, y marcar tambien las coordenadas que hay Agua para no volver a repetir un tiro
5. Siempre que vayas a hacer un tiro tienes que mirar si ya has disparado ahi, cuando lo compruebas, haces el tiro
6. Cuando tiras tienes que comprobar si hay barco o no, si hay Barco = Tocado!, si Agua = Agua y cambia el turno y si has Hundido lo muestras: Acabas de hundir un barco
7. Y cada vez que has tocado un barco: recuento de cuantos barcos te quedan y comprobar si ha acabado la partida

In [ ]:
import numpy as np
import random 

def colocar_barcos(tablero, barcos):
    """
    Se encarga de colocar los barcos en el tablero.
    Devuelve el tablero con los barcos colocados y una lista de coordenadas de los barcos.
    """
    coordenadas_de_barcos = []
    for n_barcos, size in barcos:
        coordenadas_barco = []
        while n_barcos > 0:

            if np.random.randint(0, 2, 1):
                coord1 = np.random.randint(0, 10)
                coord2 = np.random.randint(0, 10 - size + 1)

                if  "B" in tablero[max(0, coord1 - 1):min(10, coord1 + 2), max(0, coord2 - 1):min(10, coord2 + size + 2)]: #Comproba ady
                    pass
                else:
                    tablero[coord1, coord2:coord2 + size] = "B"
                    coordenadas_barco.extend([(coord1, y) for y in range(coord2, coord2 + size)]) # Este metodo te coge el iterable de esta lista comprimida y lo anade a la lista existente
                    n_barcos -= 1

            else:
                coord2 = np.random.randint(0, 10)
                coord1 = np.random.randint(0, 10 - size + 1)

                if "B" in tablero[max(0, coord1 - 1):min(10, coord1 + size + 2), max(0, coord2 - 1):min(10, coord2 + 2)]:
                    pass
                else:
                    tablero[coord1:coord1 + size, coord2] = "B"
                    coordenadas_barco.extend([(x, coord2) for x in range(coord1, coord1 + size)])
                    n_barcos -= 1

        coordenadas_de_barcos.append(coordenadas_barco)
    return tablero, coordenadas_de_barcos


def coordenadas_validas(coord_x, coord_y, tablero):
    """
    Verifica si las coordenadas entran en el tablero. 
    Toma las coordenadas y el tablero como entrada y 
    devuelve True si las coordenadas son válidas, y si no, 
    da error y devuelve False.
    """
    if coord_x < 0 or coord_x >= tablero.shape[0] or coord_y < 0 or coord_y >= tablero.shape[1]:
        print("Coordenadas incorrectas.")
        return False
    return True

def comprobar_hundido(coord_x, coord_y, coordenadas_de_barcos):
    """
    Esta función comprueba si un barco ha sido hundido dadas unas coordenadas 
    de disparo. Toma las coordenadas del disparo y la lista de coordenadas de los barcos como entrada 
    y devuelve True si el barco ha sido completamente hundido, de lo contrario, devuelve False.
    """
    for coordenadas in coordenadas_de_barcos:
        if (coord_x, coord_y) in coordenadas:
            coordenadas.remove((coord_x, coord_y))
        if len(coordenadas) == 0:
            return True
    return False

def disparar(coord_x, coord_y, tablero_ia, coordenadas_de_barcos):
    """
    Toma las coordenadas del disparo, el tablero enemigo y la lista de coordenadas
    de los barcos enemigos como entrada. Devuelve una cadena que representa 
    el resultado del disparo ('A' para agua, 'T' para tocado y hundido)
    """
    if tablero_ia[coord_x, coord_y] == " ":
        print("Agua!")
        return "A"
    if tablero_ia[coord_x, coord_y] != " ":
        if comprobar_hundido(coord_x, coord_y, coordenadas_de_barcos):
            print("¡Acabas de hundir un barco!")
            return "H" 
        print("Tocado!")
        return "T"

def disparo_ia(coordenadas_usadas_ia):
    """
    Esta función simula el disparo inteligente de la IA. Toma como entrada las coordenadas 
    previamente utilizadas por la IA y devuelve unas nuevas coordenadas de disparo.
    """
    while True:
        coord_x = random.randint(0, 9)
        coord_y = random.randint(0, 9)
        if (coord_x, coord_y) not in coordenadas_usadas_ia:
            coordenadas_usadas_ia.append((coord_x, coord_y))
            break
    return coord_x, coord_y

def comprobar_ganador(tablero_contrario):
    """
    Esta función comprueba si un jugador ha ganado la partida. Toma el tablero 
    contrario como entrada y devuelve True si todas las posiciones de los barcos 
    enemigos han sido tocadas, lo que significa que el jugador ha ganado.
    """
    contador_victoria = 0
    for numero in tablero_contrario:
        for letra in numero:
            if letra == "H":
                contador_victoria += 1
    if contador_victoria == 10:
        return True
    return False

def main():
    barcos = [(4, 1), (3, 2), (2, 3), (1, 4)]

    tablero_jugador = np.full((10, 10), " ")
    tablero_disparo_jugador = np.full((10, 10), " ")

    tablero_ia = np.full((10, 10), " ")
    tablero_disparo_ia = np.full((10, 10), " ")

    # Inicializamos los barcos
    tablero_jugador, coordenadas_de_barcos_jugador = colocar_barcos(tablero_jugador, barcos)
    tablero_ia, coordenadas_de_barcos_ia = colocar_barcos(tablero_ia, barcos)
    
    # Variables de juego
    turno = 1
    coordenadas_usadas_ia = []

    while True:
        print("\n-- TURNO", turno, "--")
        
        # Turno del jugador
        if turno % 2 != 0:
            print("Tablero del jugador:")
            print(tablero_jugador)
            print("Tablero de disparos del jugador:")
            print(tablero_disparo_jugador)
            print("Turno del jugador:")
            while True:
                coord_x = int(input("Ingrese la fila (0-9): "))
                coord_y = int(input("Ingrese la columna (0-9): "))
                if coordenadas_validas(coord_x, coord_y, tablero_disparo_jugador):
                    break
            
            resultado_disparo = disparar(coord_x, coord_y, tablero_ia, coordenadas_de_barcos_ia)
            tablero_disparo_jugador[coord_x, coord_y] = resultado_disparo
            
            if resultado_disparo == "A":  # Cambio de turno si el jugador falla o dispara a un lugar ya disparado
                turno += 1
            elif resultado_disparo == "H":
                if comprobar_ganador(tablero_disparo_ia):
                    print("¡Felicidades! ¡Has ganado!")
                    return
        
        # Turno de la IA
        else:
            print("Tablero de la IA:")
            print(tablero_ia)
            print("Tablero de disparos de la IA:")
            print(tablero_disparo_ia)
            print("Turno de la IA:")
            coord_x, coord_y = disparo_ia(coordenadas_usadas_ia)
            
            print("La IA dispara a las coordenadas:", coord_x, coord_y)
            resultado_disparo = disparar(coord_x, coord_y, tablero_jugador, coordenadas_de_barcos_jugador)
            tablero_disparo_ia[coord_x, coord_y] = resultado_disparo
            
            if resultado_disparo == "A" or resultado_disparo == "T":  # Cambio de turno si la IA falla o dispara a un lugar ya disparado
                turno += 1
            elif resultado_disparo == "H":
                if comprobar_ganador(tablero_disparo_jugador):
                    print("¡OH NO, La IA ha ganado!")
                    return

if __name__ == "__main__":
    main()

Conclusiones: 

Me he dado cuenta que cuando estamos en clase todo es mucho más fácil, lo entiendes todo, claro lo explican Jose Luis y Miguel que son unos cracks pero cuando estas en casa, solo, sin ayuda, es una historia completamente diferente y tienes que de verdad entenderlo bien, pensar y plantearlo. Y mucha prueba y error.

Una lección que he aprendido es que la charla de gestión del tiempo de dentro de unas horas me va a venir como anillo al dedo. 
Enserio, me he dado cuenta que en la vida en general pero sobre todo en esta disciplina es importantísimo gestionar bien el tiempo. 

También he aprendido que debo gestionar mejor la frustración.
Cuantas veces me habré puesto delante del ordenador, no me salía nada y he acabado siendo falsamente productivo o peor en los inframundos de YouTube?  Jajaja en fin.

Que si escribimos código y nos atascamos, en vez coger y cerrar el ordenador, intentar entender, vale porque está mal esta linea, dónde está mal, que puedo hacer para ir avanzando poco a poco. 

Que desde mi punto de vista hay que atacar el codigo con un plan y con pequeñas tomas de acción.

Esto me lleva a otra lección aprendida, la flexibilidad, oye que si estás de verdad atascado y no sabes que hacer, no tengas miedo de borrar el código y probar otra solución, otro método, otro camino… y no encoscarte de que oye esta forma, es la que he hecho yo y tiene que estar bien por cojones.

He aprendido que el trabajo en equipo es muy importante, tendría que quizás haber pedido más ayuda a mis profes, a mis compañeros, ver diferentes puntos de vista, leer diferentes códigos pero quería intentarlo hacer solo. 

Que no voy a jugar al hundir la flota nunca más 😂. Putos barquitos. Y que bueno es el primer proyecto, la hostia ya me la he metido, ya he aprendido, cuanto antes mejor, ahora a abordar los siguientes proyectos con ilusión, con otro enfoque mas proactivo, resolutivo, positivo… y a tope con los siguientes que ahora viene lo bueno!

Muchas gracias a todos